## Traer datos de amplitude


Este archivo tiene como finalidad poder extraer datos desde amplitude y poder manejarlos y moldearlos a nuestra manera antes de poder aplicarle cálculos o cualquier otro tipo de cosa a la data

In [159]:
import sys
from utils.extract_amplitude_data import start_session
from utils.df_utils import fill_na_columns
from utils.classify_ads import *
import pandas as pd

## Extracción de datos

### Datos de RQL desde Amplitude

Conociendo la data previamente, se define cómo se renombraran sus columnas y cuales se van a eliminar

In [160]:
columns_to_rename = {"Label_1": "state", "Label_2": "brand", "Label_3": "day", "Metric_1": "mx_rql_online_by_ls"}
# columns_to_fill_na = {"columns": ["rql_high_online"], "value": 0, "data_type": "float"}
columns_to_drop = ["Metric_2"]

df_rql_per_state_and_brand = start_session("codayant", columns_to_rename=columns_to_rename, columns_to_drop=columns_to_drop)

{'data': {'labels': [[], ['Estado de México', 'Bajaj', '2024-12-09T00:00:00'], ['Estado de México', 'Bajaj', '2024-12-10T00:00:00'], ['Estado de México', 'Bajaj', '2024-12-11T00:00:00'], ['Estado de México', 'Bajaj', '2024-12-12T00:00:00'], ['Estado de México', 'Bajaj', '2024-12-13T00:00:00'], ['Estado de México', 'Bajaj', '2024-12-14T00:00:00'], ['Estado de México', 'Bajaj', '2024-12-15T00:00:00'], ['Estado de México', 'Italika', '2024-12-09T00:00:00'], ['Estado de México', 'Italika', '2024-12-10T00:00:00'], ['Estado de México', 'Italika', '2024-12-11T00:00:00'], ['Estado de México', 'Italika', '2024-12-12T00:00:00'], ['Estado de México', 'Italika', '2024-12-13T00:00:00'], ['Estado de México', 'Italika', '2024-12-14T00:00:00'], ['Estado de México', 'Italika', '2024-12-15T00:00:00'], ['Estado de México', 'Honda', '2024-12-09T00:00:00'], ['Estado de México', 'Honda', '2024-12-10T00:00:00'], ['Estado de México', 'Honda', '2024-12-11T00:00:00'], ['Estado de México', 'Honda', '2024-12-12T0

In [161]:
df_rql_per_state_and_brand['day'] = pd.to_datetime(df_rql_per_state_and_brand['day']).dt.strftime('%Y-%m-%d')

In [162]:
df_rql_per_state_and_brand

,state,brand,day,mx_rql_online_by_ls
0,Estado de México,Bajaj,2024-12-09,51.0
1,Estado de México,Bajaj,2024-12-10,62.0
2,Estado de México,Bajaj,2024-12-11,61.0
3,Estado de México,Bajaj,2024-12-12,56.0
4,Estado de México,Bajaj,2024-12-13,51.0
...,...,...,...,...
1955,Coahuila,TVS,2024-12-11,1.0
1956,Coahuila,TVS,2024-12-12,0.0
1957,Coahuila,TVS,2024-12-13,0.0
1958,Coahuila,TVS,2024-12-14,0.0


## Datos de la BD de Ads

In [163]:
df_meta_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\MX BD Ads saturación mensual - Meta.csv")

In [164]:
df_state_per_amplitude = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\Geolocalización\MX equivalencias geo regiones - Equivalencias_Amplitude.csv")

In [165]:
df_state_per_media = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\Geolocalización\MX equivalencias geo regiones - Equivalencias_Medios.csv")

In [166]:
df_rql_per_state_and_brand_merged = pd.merge(df_rql_per_state_and_brand, df_state_per_amplitude, left_on="state", right_on="Amplitude_declarado")
df_rql_per_state_and_brand_merged.drop(columns = ["Amplitude_declarado"], inplace = True)

In [167]:
df_state_amplitude_and_media = pd.merge(df_rql_per_state_and_brand_merged, df_state_per_media, on= "Amplitude_ips", how = "left")

In [168]:
df_state_amplitude_and_media

,state,brand,day,mx_rql_online_by_ls,Amplitude_ips,Meta,GAds,TikTok
0,Estado de México,Bajaj,2024-12-09,51.0,México,State of Mexico,State of Mexico,Mexico
1,Estado de México,Bajaj,2024-12-10,62.0,México,State of Mexico,State of Mexico,Mexico
2,Estado de México,Bajaj,2024-12-11,61.0,México,State of Mexico,State of Mexico,Mexico
3,Estado de México,Bajaj,2024-12-12,56.0,México,State of Mexico,State of Mexico,Mexico
4,Estado de México,Bajaj,2024-12-13,51.0,México,State of Mexico,State of Mexico,Mexico
...,...,...,...,...,...,...,...,...
1955,Coahuila,TVS,2024-12-11,1.0,Coahuila,Coahuila de Zaragoza,Coahuila,Coahuila
1956,Coahuila,TVS,2024-12-12,0.0,Coahuila,Coahuila de Zaragoza,Coahuila,Coahuila
1957,Coahuila,TVS,2024-12-13,0.0,Coahuila,Coahuila de Zaragoza,Coahuila,Coahuila
1958,Coahuila,TVS,2024-12-14,0.0,Coahuila,Coahuila de Zaragoza,Coahuila,Coahuila


In [169]:
df_rqls_meta = df_state_amplitude_and_media[["day", "state", "Amplitude_ips", "Meta", "brand", "mx_rql_online_by_ls"]]
df_rqls_meta

,day,state,Amplitude_ips,Meta,brand,mx_rql_online_by_ls
0,2024-12-09,Estado de México,México,State of Mexico,Bajaj,51.0
1,2024-12-10,Estado de México,México,State of Mexico,Bajaj,62.0
2,2024-12-11,Estado de México,México,State of Mexico,Bajaj,61.0
3,2024-12-12,Estado de México,México,State of Mexico,Bajaj,56.0
4,2024-12-13,Estado de México,México,State of Mexico,Bajaj,51.0
...,...,...,...,...,...,...
1955,2024-12-11,Coahuila,Coahuila,Coahuila de Zaragoza,TVS,1.0
1956,2024-12-12,Coahuila,Coahuila,Coahuila de Zaragoza,TVS,0.0
1957,2024-12-13,Coahuila,Coahuila,Coahuila de Zaragoza,TVS,0.0
1958,2024-12-14,Coahuila,Coahuila,Coahuila de Zaragoza,TVS,0.0


In [170]:
df_rqls_meta["brand"].unique()

array(['Bajaj', 'Italika', 'Honda', 'Vento', 'CF Moto', 'Yamaha', 'KTM',
       'Benelli', 'Suzuki', 'TVS', 'Kawasaki', 'Zmoto', 'Zontes',
       'Dinamo', 'MB', 'Hero', 'Royal Enfield', 'Super Soco', 'BDS',
       'Treck'], dtype=object)

In [171]:
df_meta_ads

,Date,Campaign ID,Campaign name,Ad set ID,Ad set name,Ad ID,Ad name,Region,Impressions,Link clicks,Cost,Cost (USD)
0,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Campeche,37.0,NaN,0.82,0.04
1,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Colima,13.0,NaN,0.57,0.03
2,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Distrito Federal,335.0,2.0,9.50,0.47
3,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Durango,6.0,NaN,0.18,0.01
4,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Guanajuato,73.0,NaN,1.93,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...
36300,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115940059,LaGochaMexicanaVideo,State of Mexico,1.0,NaN,0.02,0.00
36301,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115940059,LaGochaMexicanaVideo,Tlaxcala,38.0,NaN,0.62,0.03
36302,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115950059,00000002-V1-CompraMotoEnMinutos,Puebla,133.0,4.0,4.13,0.20
36303,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115950059,00000002-V1-CompraMotoEnMinutos,State of Mexico,1.0,NaN,0.01,0.00


In [173]:
# Aplicar la función a la columna Ad name
df_meta_ads['Marca'] = df_meta_ads['Ad name'].apply(classify_ads)

In [182]:
# Convertir la columna 'Date' a datetime
df_meta_ads['Date'] = pd.to_datetime(df_meta_ads['Date'])

# Agregar columna con el número de semana (donde lunes es el primer día de la semana)
df_meta_ads['Semana'] = df_meta_ads['Date'].dt.isocalendar().week

# Agregar columna con el año
df_meta_ads['Año'] = df_meta_ads['Date'].dt.year

# Mostrar el dataframe con las nuevas columnas
df_meta_ads

,Date,Campaign ID,Campaign name,Ad set ID,Ad set name,Ad ID,Ad name,Region,Impressions,Link clicks,Cost,Cost (USD),Marca,Semana,Año
0,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Campeche,37.0,NaN,0.82,0.04,Vento,48,2024
1,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Colima,13.0,NaN,0.57,0.03,Vento,48,2024
2,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Distrito Federal,335.0,2.0,9.50,0.47,Vento,48,2024
3,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Durango,6.0,NaN,0.18,0.01,Vento,48,2024
4,2024-12-01,120211503602490059,AO | 4 | Action | ReferrerFormCompleted | MX |...,120211503602520059,ReferidosLanding,120216608352680059,VideoReferidosVento,Guanajuato,73.0,NaN,1.93,0.09,Vento,48,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36300,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115940059,LaGochaMexicanaVideo,State of Mexico,1.0,NaN,0.02,0.00,Generica,13,2025
36301,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115940059,LaGochaMexicanaVideo,Tlaxcala,38.0,NaN,0.62,0.03,Generica,13,2025
36302,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115950059,00000002-V1-CompraMotoEnMinutos,Puebla,133.0,4.0,4.13,0.20,Generica,13,2025
36303,2025-03-27,120225456115740059,AO | 1 | Abierta| RQL | Urbe Puebla| 18 + años,120225456115720059,Top Performance,120225456115950059,00000002-V1-CompraMotoEnMinutos,State of Mexico,1.0,NaN,0.01,0.00,Generica,13,2025


In [183]:
import os
from datetime import datetime

# Crear las carpetas necesarias
fecha_actual = datetime.now().strftime("%d%m%Y")
mes_actual = datetime.now().strftime("%B")
ruta_carpeta = os.path.join("./outputs/", mes_actual, fecha_actual)

os.makedirs(ruta_carpeta, exist_ok=True)

In [184]:
df_meta_ads.to_csv(os.path.join(ruta_carpeta, "df_meta_ads.csv"), index=False)

In [ ]:
import os
from datetime import datetime
fecha_actual = datetime.now().strftime("%d%m%Y")
# Crear la carpeta con el nombre de la fecha actual si no existe
carpeta_fecha_actual = f"../outputs/amplitude/{fecha_actual}"

if not os.path.exists(carpeta_fecha_actual):
    os.makedirs(carpeta_fecha_actual)

## RQLs and Deal Won combinados

In [23]:
datos_total_rql_per_day.to_csv(f"{carpeta_fecha_actual}/{fecha_actual}_data_rql_per_day.csv", index=False)